In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras import layers, losses, Input
from tensorflow.keras.models import Model

2022-05-12 14:28:34.314010: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Links:

- https://blog.keras.io/building-autoencoders-in-keras.html
- https://www.tensorflow.org/tutorials/generative/autoencoder


In [2]:
pretrain_features = pd.read_csv('handout/pretrain_features.csv')
pretrain_labels = pd.read_csv('handout/pretrain_labels.csv')
train_features = pd.read_csv('handout/train_features.csv')
train_labels = pd.read_csv('handout/train_labels.csv')
test_features = pd.read_csv('handout/test_features.csv')

## Cleaning data

In [3]:
for c in pretrain_features.columns[2:]:
    if len(pretrain_features[c].unique()) < 2:
        pretrain_features = pretrain_features.drop(c, axis=1)
        train_features = train_features.drop(c, axis=1)
        test_features = test_features.drop(c, axis=1)

## Autoencoder

In [4]:
class Autoencoder(Model):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
            Input(shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(latent_dim),
        ])
        self.decoder = tf.keras.Sequential([
            Input(shape=(latent_dim,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [5]:
X_pretrain_train, X_pretrain_val, y_pretrain_train, y_pretrain_val = train_test_split(
    pretrain_features[pretrain_features.columns[2:]], pretrain_labels[["lumo_energy"]], test_size=0.1, random_state=42
)


In [6]:
autoencoder = Autoencoder(X_pretrain_train.shape[1], latent_dim=32)
autoencoder.compile(optimizer='adam', loss=losses.MSE)

2022-05-12 14:29:06.186506: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-12 14:29:06.187907: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 14:29:06.187944: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-12 14:29:06.187988: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t3ui01.psi.ch): /proc/driver/nvidia/version does not exist
2022-05-12 14:29:06.188590: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild Ten

In [7]:
history = autoencoder.fit(X_pretrain_train, X_pretrain_train, epochs=10, batch_size=32, validation_data=(X_pretrain_val, X_pretrain_val))

2022-05-12 14:29:13.187379: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-12 14:29:13.187949: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294660000 Hz


Epoch 1/10
1407/1407 [==============================] - 5s 3ms/step - loss: 0.0224 - val_loss: 0.0141
Epoch 2/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0136 - val_loss: 0.0123
Epoch 3/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0120 - val_loss: 0.0113
Epoch 4/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0110 - val_loss: 0.0106
Epoch 5/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0104 - val_loss: 0.0103
Epoch 6/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0100 - val_loss: 0.0099
Epoch 7/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0096 - val_loss: 0.0096
Epoch 8/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0093 - val_loss: 0.0093
Epoch 9/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0091 - val_loss: 0.0092
Epoch 10/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.009

## PreModel

In [8]:
class ELumon(Model):
    def __init__(self, autoencoder):
        super(ELumon, self).__init__()
        self.encoder = autoencoder.encoder
        self.regressor = tf.keras.Sequential([
            Input(shape=self.encoder.output_shape[1:]),
            layers.BatchNormalization(),
            layers.Dense(1, activation='linear', kernel_regularizer="l2")
        ])
        # Set all layers to trainable
        for layer in self.encoder.layers:
            layer.trainable = False

    def call(self, x):
        encoded = self.encoder(x)
        return self.regressor(encoded)

In [16]:
elumon = ELumon(autoencoder)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
elumon.compile(optimizer=opt, loss="mean_squared_error")

In [17]:
elumon.fit(X_pretrain_train, y_pretrain_train, epochs=2, batch_size=1000, validation_data=(X_pretrain_val, y_pretrain_val))

Epoch 1/2
45/45 [==============================] - 1s 11ms/step - loss: nan - val_loss: nan
Epoch 2/2
45/45 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan


In [144]:
t = elumon.predict(X_pretrain_val[:2])

In [145]:
t

array([[4.7697188e-08],
       [2.7368214e-08]], dtype=float32)